Author: Amy Weng

This code is for running named entity recognition (NER) using spaCy on individual Restoration-era texts that mention monopoly, East India Company, and other terms we are interested in. 

FIXES: I filtered out texts with the same title from my eicFolder using "east india company|east-india company|east-india-company" this time. I got much more results after including the latter two, which emphasizes the importance of researchers knowing the linguistic conventions of the time. 

---------------------
CHALLENGES: I tried to run the algorithm on the entire monopolyFolder, but NER w/ spaCy has a max character limit of 1 million (see last cell on this page). The code ran for nearly 20 minutes before failing. What I tried next was to filter through the texts further for mentions of the East India Company to narrow down the number of texts. 

In [1]:
import os
import re
import pandas as pd
import string
import sys
# !{sys.executable} -m pip install spacy
# !{sys.executable} -m spacy download en_core_web_sm
import spacy

directory = r'C:\Users\amycw\Desktop\ecbc research'
monopolyFolder = directory + "\\" + 'monopoly_eebo'
eicFolder = directory + "\\"+'eic_monopoly_eebo'

In [2]:
eicCount = 0;
eicFile = open(r'C:\Users\amycw\Desktop\ecbc research\eic titles and filenames.txt', "a")

for name in os.listdir(monopolyFolder):
    readFile = pd.read_csv(monopolyFolder + "\\" + name)
    num_rows = readFile.index
    num_texts = len(num_rows)
    for i in range(num_texts):
        data = readFile[i:(i+1)]
        text = data.iloc[0].text
        title = data.iloc[0].title
        eic = re.compile("east india company|east-india company|east-india-company")
        if (re.search(eic, text) != None):
            moveFile = eicFolder + "\\" + name
            data.to_csv(moveFile, index=False)
            eicFile.write(name + "\n" + title +"\n")
            eicCount +=1
print("the number of texts that mention the EIC is "+str(eicCount)+"\n")
eicFile.close()

the number of texts that mention the EIC is 61



In [3]:
# medCount = 0; 
allMedTerms = []
medCount = 0
medFile = open(r'C:\Users\amycw\Desktop\ecbc research\medical_eic_monopoly.txt', "a")
for name in os.listdir(eicFolder):
    readFile = pd.read_csv(eicFolder + "\\" + name)
    num_rows = readFile.index
    num_texts = len(num_rows)
    for i in range(num_texts):
        data = readFile[i:(i+1)]
        text = data.iloc[0].text
        lexicon = re.compile("corruption|consumption|body politick|corrupt|consume|wasting|waste|blood|physician|decay|body|canker|cancer|disease|illness|remedy|cure|sickness|hepatitis|fever|spirit|brain|mind|vital|therapeutic|degeneration|degenerate|plague|smells|putrid|bad|tetrid|breathing|wholesome|healthy|unhealthy|sane|insane|nervous|languish|faculties|enfeeblement|drinks|tuberculosis|constitution|bile|black bile|yellow bile|phlegm|lung|sores|fog|smoke|diagnosis|prognosis|fiber|atrophy|morbid|mortal|mortality|nerves|inanition|defect|distemper|swelling|upset|stomach|cough|exercise|unwholesome|evacuation|fatal|fatality|vessels|hemmorrhage|bleeding|bleed|melancholy|diabetes|asthma|vomit|opiate|opium|ulcer|envy|jealousy|spoil|liver|vein|supple|heart|mouth|cured|pox")
        if (re.search(lexicon, text) != None):
            for w in (re.findall(lexicon, text)):
                allMedTerms.append(w)
                medFile.write(w+" ")
        medCount+=1
print("the number of files that have medical terms is "+str(medCount))
medFile.close()

the number of files that have medical terms is 61


In [4]:
uniqueMed = {} 
terms = []
for w in allMedTerms:
    if w not in terms: 
        uniqueMed[w] = 1
        terms.append(w)
    else: 
        uniqueMed[w] = uniqueMed[w] + 1 

for pair in sorted (uniqueMed.items(), key=lambda kv:(kv[1], kv[0]), reverse=True):
    print(str(pair))

('cure', 896)
('body', 720)
('liver', 542)
('blood', 482)
('spirit', 376)
('mind', 369)
('heart', 284)
('bad', 266)
('mouth', 252)
('constitution', 235)
('remedy', 227)
('consumption', 165)
('disease', 159)
('exercise', 149)
('consume', 134)
('spoil', 128)
('mortal', 121)
('physician', 116)
('defect', 112)
('waste', 100)
('decay', 100)
('plague', 97)
('fatal', 89)
('corrupt', 74)
('corruption', 60)
('melancholy', 58)
('jealousy', 55)
('brain', 47)
('sickness', 45)
('languish', 44)
('degenerate', 44)
('fever', 41)
('distemper', 41)
('stomach', 35)
('vessels', 32)
('supple', 32)
('wasting', 30)
('envy', 29)
('lung', 26)
('wholesome', 25)
('vein', 23)
('bleeding', 19)
('breathing', 12)
('vital', 11)
('smoke', 11)
('pox', 11)
('bile', 11)
('swelling', 10)
('fog', 8)
('cough', 8)
('bleed', 8)
('vomit', 7)
('sores', 7)
('smells', 7)
('nerves', 7)
('faculties', 7)
('drinks', 5)
('sane', 4)
('canker', 4)
('unwholesome', 3)
('opium', 3)
('putrid', 2)
('phlegm', 2)
('cancer', 2)
('healthy', 1)
(

In [24]:
# check for files that are bigger than 1000000 
for file in os.listdir(eicFolder):
    entries = pd.read_csv(eicFolder + "\\" + file)
    num_rows = entries.index
    num_texts = len(num_rows)
    for i in range(num_texts):
        data = entries[i:(i+1)]
        name = tooBig + "\\" + file
        text = data.iloc[0].text
        title = data.iloc[0].title
        if (len(text)>=1000000):
            print(title+"\n")
            print(len(text))

the history of england giving a true and impartial account of the most considerable transactions in church and state in peace and war during the reigns of all the kings and queens from the coming of julius caesar into britain with an account of all plots conspiracies insurrections and rebellions likewise a relation of the wonderful prodigies to the year together with a particular description of the rarities in the several counties of england and wales with exact maps of each county by john seller 

1185040


In [33]:
#running named entity recognition using spaCy on eicFolder (45 texts in total)
#took 13 minutes 
nerFile = open(r'C:\Users\amycw\Desktop\ecbc research\namedEntityRecognition.txt', "a")
ner = spacy.load('en_core_web_sm')
for fileName in os.listdir(eicFolder):
    readFile = pd.read_csv(eicFolder + "\\" + fileName)
    num_texts = len(readFile.index)
    for i in range(num_texts):
        data = readFile[i:(i+1)]
        text = data.iloc[0].text
        entities = ner(text)
        for ent in entities.ents:
            nerFile.write(ent.text+" "+ent.label_+"\n")
nerFile.close()

In [34]:
# alphabetize and sort entities by frequency

nerFile = open(r'C:\Users\amycw\Desktop\ecbc research\namedEntityRecognition.txt', "r")
SortedNER = open(r'C:\Users\amycw\Desktop\ecbc research\sortedNamedEntityRecognition.txt',"a")
SortedNER.write("format is (<name> <TYPE>, <frequency>)\n")
uniqueEnts = {} 
entities = []
indices = []
entities = nerFile.read().split("\n")
for ent in entities:
    if ent not in indices: 
        uniqueEnts[ent] = 1
        indices.append(ent)
    else: 
        uniqueEnts[ent] = uniqueEnts[ent] + 1 
alphabetizedEnts = {}
for ent in sorted(uniqueEnts): 
    alphabetizedEnts[ent]=uniqueEnts[ent] 
for pair in sorted (alphabetizedEnts.items(), key=lambda kv:(kv[1], kv[0]), reverse=True):
    SortedNER.write(str(pair)+"\n")
SortedNER.close()

In [25]:
#break the too big file into two smaller ones 

breakFile = r'C:\Users\amycw\Desktop\ecbc research\eic_monopoly_eebo\A5_P4_3070.csv'
entries= pd.read_csv(breakFile)
half = []
otherHalf = []
count = 0 
string = ""
other = ""
for i in range(len(entries.index)):
    d = entries[i:(i+1)]
    text = d.iloc[0].text
    words = text.split(" ")
    for w in words: 
        if (count <= (len(words)/2)):
            half.append(w)
            count+=1
        else:
            otherHalf.append(w)
for w in half:
    string = string + w + " "
for w in otherHalf: 
    other = other + w + " "

df1 = pd.DataFrame([(d.iloc[0].index, d.iloc[0].title,d.iloc[0].author,d.iloc[0].publisher,d.iloc[0].date,string)],
           columns=('index', 'title','author','publisher','date','text'))
df2 = pd.DataFrame([(d.iloc[0].index, d.iloc[0].title,d.iloc[0].author,d.iloc[0].publisher,d.iloc[0].date,other)],
           columns=('index', 'title','author','publisher','date','text'))
df1.to_csv(eicFolder + "\\" + 'A5_P4_3070_1.csv', index=False)
df2.to_csv(eicFolder + "\\" + 'A5_P4_3070_2.csv', index=False)

In [34]:
# failed attempt 
nerFile = open(r'C:\Users\amycw\Desktop\ecbc research\monopoly_NER.txt', "a")
nerFile.write("format is (name type, frequency)\n")

ner = spacy.load('en_core_web_sm')
uniqueEnts = {} 
for fileName in os.listdir(monopolyFolder):
    readFile = pd.read_csv(monopolyFolder + "\\" + fileName)
    num_rows = readFile.index
    num_texts = len(num_rows)
    for i in range(num_texts):
        data = readFile[i:(i+1)]
        text = data.iloc[0].text
        entities = ner(text)
        for ent in entities.ents:
            idx = ent.text+" "+ent.label_
            if ent not in uniqueEnts: 
                uniqueEnts[idx] = 1
            else: 
                uniqueEnts[idx] += 1 
alphabetizedEnts = {}
for ent in sorted(uniqueEnts): 
    alphabetizedEnts[ent]=uniqueEnts[ent] 
for pair in sorted (alphabetizedEnts.items(), key=lambda kv:(kv[1], kv[0]), reverse=True):
    nerFile.write(str(pair)+"\n")
nerFile.close()

ValueError: [E088] Text of length 1449451 exceeds maximum of 1000000. The parser and NER models require roughly 1GB of temporary memory per 100,000 characters in the input. This means long texts may cause memory allocation errors. If you're not using the parser or NER, it's probably safe to increase the `nlp.max_length` limit. The limit is in number of characters, so you can check whether your inputs are too long by checking `len(text)`.